In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
####Path to code user needs to update these three parameters#####
path_to_code = "/Users/mischlv/Desktop/PhD/Pipeline/myPPI_Dataset_04_27_v2/CSV_files_08_06/myPPI_Protein_model_5_xgboost_SBS_unbalanced_0.25_6featsfromothermodel_hp.pkl"
#####path_to_model_code_folder is the path to where the AF models are stored. Does not have to be the same location as the code
path_to_AF_models = "/Users/mischlv/Desktop/PhD/Pipeline/myPPI_Peptide_Dataset_wtemps/Peptide_Dataset_08_03"
#### name of feature_file####
name_of_feature_csv_file = "myPPI_Peptide_08_03_fixed"

In [3]:
df = pd.read_csv(f"{path_to_AF_models}/{name_of_feature_csv_file}.csv")

columns_red_feats =  [2, 4, 5, 8, 9, 15]
df_columns_red_feats = [-1, 2, 4, 5, 8, 9, 15,]
columns_red_feats_std = [val + 1 for val in columns_red_feats]
df_columns_red_feats_std = [val + 1 for val in df_columns_red_feats]

nonans_red_feats = df.iloc[:, columns_red_feats_std].dropna()
df_nonans_red_feats = df.iloc[:, df_columns_red_feats_std].dropna()


X = nonans_red_feats.values
X_df = df_nonans_red_feats

In [4]:
print(X_df)

                                              File_name  ipLDDT_12A  \
0     249_5LZ6_205.0_wtemps_relaxed_rank_001_alphafo...       93.52   
1     191_3DXD_176.0_wtemps_relaxed_rank_001_alphafo...       95.19   
2     182_1MXE_174.0_wtemps_relaxed_rank_002_alphafo...       91.37   
3     221_4BWQ_186.0_wtemps_relaxed_rank_002_alphafo...       97.50   
4     259_5TWA_213.0_wtemps_relaxed_rank_004_alphafo...       96.63   
...                                                 ...         ...   
4549  386_2I3S_2I3S_chainA_3ZGA_chainB_decoy_397.0_r...       54.65   
4550  403_4NM0_4NM0_chainA_6TQP_chainB_decoy_416.0_r...       67.07   
4551  365_2UZV_2UZV_chainA_1AVF_chainB_decoy_363.0_r...       58.37   
4552  372_1AVF_1AVF_chainA_3ZGA_chainB_decoy_377.0_r...       49.30   
4553  373_5LI1_5LI1_chainA_2MNJ_chainB_decoy_378.0_r...       57.42   

      iPAE_top_1_4_12A  tPAE_12A  BE_bining_energy_12A  fa_atr_complex_12A  \
0               1.0604    9.0979           -109.321368        -1033.1

In [5]:
# Load the Xgboost model
final_model = joblib.load(f"{path_to_code}")
# Use the loaded model and threshold for predictions
model_scores = final_model.predict_proba(X)[:, 1]
# Load the custom threshold from metadata
threshold = 0.5
predicted_label = (model_scores >= threshold).astype(int)

In [7]:
# Create the initial results dataframe
results = pd.DataFrame({
    'File_name': X_df['File_name'],
    'score': model_scores,
    'predicted_label': predicted_label
})

def label_to_prediction(label):
    return 'interacting' if label == 1 else 'not interacting'

results['prediction'] = results['predicted_label'].apply(label_to_prediction)
###Group all 5 models together based on name
results['common_name'] = results['File_name'].apply(lambda x: x.split("_relaxed_")[0])

# Rank by score within each common_name group
results['prob_rank'] = results.groupby('common_name')['score'].rank(method='first', ascending=False).astype(int)

# Filter rows where prob_rank is 1
filtered_df = results[results['prob_rank'] == 1]

final_df = filtered_df.drop(columns=['prob_rank', 'common_name'])
print(final_df)

                                              File_name     score  \
0     249_5LZ6_205.0_wtemps_relaxed_rank_001_alphafo...  0.986439   
2     182_1MXE_174.0_wtemps_relaxed_rank_002_alphafo...  0.986208   
3     221_4BWQ_186.0_wtemps_relaxed_rank_002_alphafo...  0.985202   
4     259_5TWA_213.0_wtemps_relaxed_rank_004_alphafo...  0.986102   
5     215_3R85_183.0_wtemps_relaxed_rank_005_alphafo...  0.984989   
...                                                 ...       ...   
4519  358_3HYM_3HYM_chainA_1PJM_chainB_decoy_351.0_r...  0.095182   
4530  401_3ZWL_3ZWL_chainA_1QGK_chainB_decoy_414.0_r...  0.374241   
4536  369_2NUD_2NUD_chainA_1SP4_chainB_decoy_372.0_r...  0.201177   
4547  372_1AVF_1AVF_chainA_3ZGA_chainB_decoy_377.0_r...  0.048635   
4548  382_2VUT_2VUT_chainA_3VUX_chainB_decoy_388.0_r...  0.074972   

      predicted_label       prediction  
0                   1      interacting  
2                   1      interacting  
3                   1      interacting  
4      

In [8]:
####The score is in the column score and the predicted label based on a 0.5 threshold is under the column predicted_label###
filtered_df.to_csv(f"{path_to_AF_models}/myPPI_afv3_6featfromothermodel_peptide_dataset.csv", mode='w', header=True, index=False)

